# 层类

**层**，是深度神经网络的一个重要逻辑概念。每一层包含若干人工神经元；每个人工神经元拥有自己的参数（权重和偏置），和加工、传输数据的能力。

在我们第一部分的代码中，这些参数和逻辑分散在不同的位置，需要我们手动管理它们的创建和调用。随着网络变深，我们需要手动管理成百上千个这样的变量和函数，非常容易出错。

为了解决这个难题，我们将引入**层**（Layer）类，作为封装人工神经元的参数和逻辑函数的容器。

层也将成为**前向传播链路**的主要载体。

In [26]:
from abc import abstractmethod, ABC
import numpy as np

<div style="border-left: 4px solid #4CAF50; background:#f9f9f9; padding:10px; margin:10px 0;">
<strong>💡 提示：</strong> <strong>ABC</strong> 是 Python 的抽象基类组件，可以用它以继承的方式定义抽象基类。
</div>

## 基础架构

### 张量

我们为张量添加了一个新的属性：**大小**（size），用来获得张量数据的数值个数。

In [27]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)

    @property
    def size(self):
        return len(self.data)

    def __repr__(self):
        return f'Tensor({self.data})'

### 基础层

基础层是一个抽象类，定义了一个**前向传播**（forward）的虚拟接口。这个接口需要实现的功能，是对输入数据进行加工，然后把结果作为输出数据传递给下一层。

网络模型所有的层，通过这个接口连接在一起，就构成了网络模型的前向传播链路。

In [28]:
class Layer(ABC):

    def __call__(self, x: Tensor):
        return self.forward(x)

    @abstractmethod
    def forward(self, x: Tensor):
        pass

    def __repr__(self):
        return ''

## 数据

### 特征

In [29]:
feature = Tensor([28.1, 58.0])

## 模型

### 线性层

我们定义的第一个层类是**线性层**（Linear Layer），用来封装人工神经元的基本功能：线性回归。

线性层同时封装了对应的参数：权重和偏置，也可以认为是封装了本层的模型状态。

线性层的功能等同于一组人工神经元，权重数组的每一行代表一个人工神经元。

创建一个线性层，我们需要知道：

* in_size：（输入）特征值的数量；
* out_size：（输出）预测值的数量。

```{figure} images/layer-weight.png
:align: center
:width: 480px
**图例：线性层结构**
```

* $x_1、x_2$：特征值
* $w_{n.1}、w_{n.2}、b_n$：第 $n$ 个人工神经元的权重和偏置
* $p_1、p_2、p_3、p_4$：预测值

In [30]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        self.weight = Tensor(np.ones((out_size, in_size)) / in_size)
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        return Tensor(x.data @ self.weight.data.T + self.bias.data)

    def __repr__(self):
        return f'Linear[weight{self.weight.data.shape}; bias{self.bias.data.shape}]'

## 验证

### 建模

我们需要创建一个只有一个人工神经元的线性层；这个人工神经元可以处理 2 个（输入）特征值，并产生 1 个（输出）预测值。

In [31]:
layer = Linear(feature.size, 1)
print(layer)

Linear[weight(1, 2); bias(1,)]


### 推理

In [32]:
prediction = layer(feature)
print(f'prediction: {prediction}')

prediction: Tensor([43.05])


通过层的概念，我们实现了模型前向传播的功能。